In [ ]:
# -*- coding: utf-8 -*-

###Using Chunking, we split the video into several segments

import speech_recognition as sr 
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip


def with_moviepy(filename):
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(filename)
    duration       = clip.duration
    fps            = clip.fps
    width, height  = clip.size
    return duration, fps, (width, height)

os.chdir('your_location')
_path=('your_location')
print(with_moviepy(_path+'videoplayback.mp4'))

num_seconds_video= round(649.42)
print("The video is {} seconds".format(num_seconds_video))
l=list(range(0,num_seconds_video+1,60))
print(l)

diz={}
for i in range(len(l)-1):
    ffmpeg_extract_subclip("videoplayback.mp4", l[i]-2*(l[i]!=0), l[i+1], targetname="chunks/cut{}.mp4".format(i+1))
    clip = mp.VideoFileClip(r"chunks/cut{}.mp4".format(i+1)) 
    clip.audio.write_audiofile(r"converted/converted{}.wav".format(i+1))
    r = sr.Recognizer()
    audio = sr.AudioFile("converted/converted{}.wav".format(i+1))


In [15]:
##With individual WAV file, we split the file further into 1 MB subsegments and get the text

import speech_recognition as sr 
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def extract_text_from_wav(wav_file, language="en-GB"):
    """Extracts text from a WAV file in British English.

    Args:
        wav_file (str): The path to the WAV file.
        language (str): The language of the audio file.

    Returns:
        str: The text extracted from the WAV file.
    """

    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language=language)
    except sr.UnknownValueError:
        text = "Unknown"

    return text

def split_wav(wav_file, output_dir):
    """Splits a large WAV file into 1MB small files.

    Args:
        wav_file (str): The path to the WAV file.
        output_dir (str): The directory to save the small files.
    """

    wav = wave.open(wav_file, "rb")
    num_chunks = int(wav.getnframes() / 1048576)

    for i in range(num_chunks):
        chunk_start = i * 1048576
        chunk_end = min((i + 1) * 1048576, wav.getnframes())
        chunk_data = wav.readframes(chunk_end - chunk_start)

        chunk_file = os.path.join(output_dir, f"chunk{i:05}.wav")
        with open(chunk_file, "wb") as f:
            f.write(chunk_data)



def transcribe_large_wav(wav_file, language="en-GB"):
    """Transcribes a large WAV file in British English by splitting it into smaller chunks.

    Args:
        wav_file (str): The path to the WAV file.
        language (str): The language of the audio file.

    Returns:
        str: The text extracted from the WAV file.
    """
    
    chunks = []
    with open(wav_file, "rb") as f:
        chunk_size = 1024 * 1024  # 1MB
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            chunks.append(chunk)

    text = ""
    for chunk in chunks:
        text += extract_text_from_wav(chunk, language=language)

    return text